In [ ]:
import os

In [ ]:
emotions = { '01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fear',
            '07':'disgust','08':'surprise'}

In [ ]:
import librosa

In [ ]:
def extract_features(file_path, features=("mfcc", "mel")):
  y, sr = librosa.load(file_path)  # Load audio and get sampling rate
  feature_vector = []
  for feature in features:
    if feature == "mfcc":
      mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
      feature_vector.append(np.mean(mfccs.T, axis=0))  # Average across time
    elif feature == "mel":
      melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
      feature_vector.append(np.mean(melspectrogram.T, axis=0))  # Average across time
    elif feature == "chroma":
      stft = np.abs(librosa.stft(y))
      chroma_features = librosa.feature.chroma_stft(S=stft, sr=sr)
      feature_vector.append(np.mean(chroma_features.T, axis=0))
    else:
      raise ValueError(f"Unsupported feature: {feature}")

  return np.concatenate(feature_vector)

# Example usage
features = extract_features("path/to/audio.wav", features=("mfcc", "mel"))


